In [ ]:
# https://carla.readthedocs.io/en/0.9.12/adv_agents/#:~:text=set_destination(end_location%2C%20start_location%3DNone,use%20the%20current%20agent%20location
#BEHAVIOUR AGENT CON SETTAGGIO DESTINAZIONE

# per generazione traffico mediante agenti

#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
import threading

from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

# GENERA NV AUTO CASUALI ABILITANDONE L'AGENT BEHAVIOURAL
NV = 15

agent_list = []
vehicle_list = []

def draw_symbol(wp, duration, symbol, r, g, b):
    world.debug.draw_string(wp.transform.location, symbol, draw_shadow=False, color=carla.Color(r, g, b), life_time=duration)

#colora di bianco il percorso dato in input
def roam_vehicle(vehicle):
    #RECUPERO I WAYPOINTS CORRISPONDENTI RISPETTIVAMENTE ALLE COORDINATE DI SPWN DI SPAWN DEL VEICOLO E DI DESTINAZIONE DELL'AGENTE
    start_location = vehicle.get_location()
    start_waypoint = world.get_map().get_waypoint(start_location)
    
    #PUNTO A CASO DALLA MAPPA COME DESTINAZIONE, SCEGLIE DI DEFAULT IL PERCORSO PIU' BREVE VERSO QUEL PUNTO
    dest_location = random.choice(spawn_points).location
    dest_waypoint = world.get_map().get_waypoint(dest_location)
    
    #L' AGENTE BEHAVIOUR REAGISCE A SEMAFORI E SEGNALI DI STOP, EVITA PEDONI, SEGUE LE MACCHINE 
    #3 POSSIBILI PROFILI AL MOMENTO: cautious, normal, aggressive -> https://carla.readthedocs.io/en/0.9.12/adv_agents/#behavior-types
    agent = BehaviorAgent(vehicle, behavior='cautious')
    
    #RECUPERA IL PERCORSO CHE SEGUIRA' L'AGENTE IN UNA LISTA DI WAYPOINTS
    route = agent.trace_route(start_waypoint, dest_waypoint)
    
    #DESTINAZIONE AGENT
    agent.set_destination(dest_location)
    #lista vehicolo agent
    agent_list.append(agent)
    
    print(f"Avvio agente per veicolo: {vehicle}")
    '''spectator = world.get_spectator() 
    transform = carla.Transform(first_v.get_transform().transform(carla.Location(x=-4,z=2.5)),first_v.get_transform().rotation) 
    spectator.set_transform(transform)'''

    #MAIN LOOP:
    while True:
        if agent.done(): #SE L'AGENT HA CONCLUSO COLORA DI BIANCO IL PERCORSO COMPLETATO
            print(f"Il veicolo {npc_vehicle} ha raggiunto la sua destinazione")
            vehicle.destroy()
            break
        #APPLICA IL CONTROLO DELL'AGENTE
        vehicle.apply_control(agent.run_step())
        curr_wp = world.get_map().get_waypoint(vehicle.get_location())
        draw_symbol(curr_wp, 0.01, '!!!CAR!!!', 0, 0, 255) 

######################################################################################################################################################
#########################################################            MAIN CODE               #########################################################
######################################################################################################################################################

# CONNESSIONE AL SIMULATORE
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

#RECUPERA ASSETS DEL SIMULATORE
world = client.get_world()
bp_lib = world.get_blueprint_library() 

#TUTTI I PUNTI DI SPAWN (CONSIGLIATI) DELLA MAPPA
spawn_points = world.get_map().get_spawn_points()

first_v = None
npc_vehicle = None

for i in range(NV):
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc_vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    
    if i == 0: 
        first_v = npc_vehicle
        
    if npc_vehicle != None :
        print(f"veicolo n {i}")
        print(npc_vehicle)
        vehicle_list.append(npc_vehicle)

# Creazione e avvio dei thread per ciasucn agente
threads = []
for i in vehicle_list:
    # Crea un thread per ogni veicolo
    thread = threading.Thread(target=roam_vehicle, args=(i,))
    threads.append(thread)
    thread.start()

# Attende che tutti i thread terminino
for thread in threads:
    thread.join()



print("Programma concluso")

veicolo n 0
Actor(id=105, type=vehicle.tesla.model3)
veicolo n 1
Actor(id=106, type=vehicle.mitsubishi.fusorosa)
veicolo n 2
Actor(id=107, type=vehicle.mini.cooper_s_2021)
veicolo n 3
Actor(id=108, type=vehicle.audi.tt)
veicolo n 4
Actor(id=109, type=vehicle.tesla.model3)
veicolo n 5
Actor(id=110, type=vehicle.carlamotors.carlacola)
veicolo n 6
Actor(id=111, type=vehicle.audi.tt)
veicolo n 7
Actor(id=112, type=vehicle.mini.cooper_s_2021)
veicolo n 8
Actor(id=113, type=vehicle.audi.a2)
veicolo n 9
Actor(id=114, type=vehicle.mercedes.sprinter)
veicolo n 10
Actor(id=115, type=vehicle.ford.mustang)
veicolo n 12
Actor(id=116, type=vehicle.micro.microlino)
veicolo n 13
Actor(id=117, type=vehicle.mercedes.sprinter)
veicolo n 14
Actor(id=118, type=vehicle.lincoln.mkz_2020)
Avvio agente per veicolo: Actor(id=106, type=vehicle.mitsubishi.fusorosa)
Avvio agente per veicolo: Actor(id=108, type=vehicle.audi.tt)
Avvio agente per veicolo: Actor(id=109, type=vehicle.tesla.model3)
Avvio agente per veic